In [1]:
from numba import njit, prange
import time 



In [6]:
@njit(nopython=True, parallel=True)
def test_yesNumba():
    total = 0
    for i in prange(10**8):
        total += i + 1
    return total

def test_noNumba():
    for i in range(10**8):
        a = i + 1
    return a

#now also test teh same test but using numpy first with numba and tehn without
import numpy as np
def test_numpy_noNumba():
    a = np.arange(10**8)
    a = a + 1
    return a

@njit(nopython=True)
def test_numpy_yesNumba():
    a = np.arange(10**8)
    a = a + 1
    return a


c:\Users\pierg\Desktop\VirtualEnvironments\Standard_env\Lib\site-packages\numba\core\decorators.py:246: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


In [7]:
#test how much faster it is to use numba and print teh time

start = time.time()
test_noNumba()
end = time.time()
print("Time taken in seconds without numba: ", end - start)

start = time.time()
test_yesNumba()
end = time.time()   
print("Time taken in seconds with numba: ", end - start)

start = time.time()
test_numpy_noNumba()
end = time.time()
print("Time taken in seconds without numba and using numpy: ", end - start)

start = time.time()
test_numpy_yesNumba()
end = time.time()
print("Time taken in seconds with numba and using numpy: ", end - start)



Time taken in seconds without numba:  4.90257453918457
Time taken in seconds with numba:  0.5532357692718506
Time taken in seconds without numba and using numpy:  0.32908201217651367
Time taken in seconds with numba and using numpy:  0.7844164371490479


In [ ]:

def calculate_bpd_matrix(population):
    """
    Efficiently calculates the Broken Pair Distance (BPD) for a population of TSP solutions
    without explicit loops, using NumPy's broadcasting features.
    
    Args:
    - population (numpy.ndarray): A 2D array where each row is a TSP solution (a permutation of cities).
    
    Returns:
    - bpd_matrix (numpy.ndarray): A matrix of BPD values between each pair of solutions in the population.
    """
    
    # Number of solutions (rows) and cities (columns) in the population
    num_solutions, num_cities = population.shape
    
    # Create edge pairs for each solution using roll (shift)
    edges = np.column_stack((population, np.roll(population, -1, axis=1)))  # (city, next_city) pairs
    
    # Broadcast edges of all pairs of solutions to compare them
    # Using broadcasting to compare all pairs of solutions in a vectorized way
    edges_i = edges[:, None, :]  # Shape: (num_solutions, 1, num_cities)
    edges_j = edges[None, :, :]  # Shape: (1, num_solutions, num_cities)
    
    # Compare the edges between all pairs: where edges are different, we get a `True` (1), else `False` (0)
    differences = (edges_i != edges_j).astype(int)
    
    # Sum the differences to get the BPD for all pairs
    bpd_matrix = np.sum(differences, axis=2)  # Sum over the edge dimension (cities)
    
    return bpd_matrix

def average_bpd(population):
    """
    Calculate the average Broken Pair Distance (BPD) for each solution in the population.
    
    Args:
    - population (numpy.ndarray): 2D array where each row is a solution (TSP route).
    
    Returns:
    - avg_bpd (numpy.ndarray): Array of average BPD for each solution in the population.
    """
    bpd_matrix = calculate_bpd_matrix(population)
    avg_bpd = np.mean(bpd_matrix, axis=1)  # Calculate average BPD for each solution
    return avg_bpd


def fitness_function_calculation(population, weight_distance, weight_bdp, distance_matrix):
    '''
    - Calculate the fitness of the population based on the distance and the broken pair distance.
    
    Args:
    - population (numpy.ndarray): 2D array where each row is a TSP solution (a permutation of cities).
    - weight_distance (float): Weight factor for distance contribution to fitness.
    - weight_bdp (float): Weight factor for BPD contribution to fitness.
    - distance_matrix (numpy.ndarray): Precomputed distance matrix for the cities.
    
    Returns:
    - fitness (numpy.ndarray): Array of fitness values for each solution in the population.
    '''
    # Step 1: Calculate the distance fitness
    distance_fitness = calculate_distance_numpy(population, distance_matrix)
    
    # Step 2: Calculate the BPD fitness (average BPD for each solution)
    avg_bpd = average_bpd(population)
    
    # Step 3: Calculate the fitness using both distance and BPD
    fitness = 1 / (weight_distance * distance_fitness) + weight_bdp * avg_bpd
    
    return fitness, distance_fitness, avg_bpd




